## import

In [16]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
from torchvision import transforms
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm

In [ ]:
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

## data loader

In [17]:
class ButterflyMothDataset(Dataset):
    def __init__(
        self,
        data_folder="data",
        csv_file="butterflies and moths.csv",
        dataset="train",
    ):
        self.data_folder = data_folder
        self.df = pd.read_csv(os.path.join(self.data_folder, csv_file))
        self.df = self.df[self.df["data set"] == dataset]
        self.transform = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    def __getitem__(self, index):
        item = self.df.iloc[index]
        image = Image.open(os.path.join(self.data_folder, item["filepaths"])).convert(
            "RGB"
        )
        image = self.transform(image)
        class_id = item["class id"]
        label = item["labels"]
        return image, class_id

    def __len__(self):
        return len(self.df)


class ButterflyMothLoader(DataLoader):
    def __init__(self, dataset="train", **kwargs):
        self.dataset = ButterflyMothDataset(dataset=dataset)
        super().__init__(dataset=self.dataset, **kwargs)


train_loader = ButterflyMothLoader("train", batch_size=32, shuffle=True)
test_loader = ButterflyMothLoader("test", batch_size=32, shuffle=False)
valid_loader = ButterflyMothLoader("valid", batch_size=32, shuffle=False)

## model

In [18]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.features = nn.Sequential(
            # 第一塊
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 第二塊
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 第三塊
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 第四塊
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 第五塊
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x

model = VGG19()

## optimizer

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## train

In [20]:
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        # 封裝 train_loader 到 tqdm 進度條中
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            # 清空梯度
            optimizer.zero_grad()

            # 前向傳播
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 反向傳播和優化
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            # 更新進度條的描述
            progress_bar.set_description(f'Epoch {epoch + 1}/{num_epochs} Loss: {running_loss / (progress_bar.n + 1)}')
        print(f'Epoch {epoch+1}, Average Loss: {running_loss/len(train_loader)}')


# 設定設備
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 開始訓練
train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10:   0%|          | 0/12594 [00:00<?, ?it/s]

KeyboardInterrupt: 